# DIS21a 1 | Big Data | Exam WS 2022/23 | Group J (10)

## Participants

-   [Markus Hardtke](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2705)
    -   Matriculation Number: 1234567
-   [Furkan Erdogan](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2688)
    -   Matriculation Number: 1234567
-   [Jannik Loose](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2687)
    -   Matriculation Number: 1234567
-   [Gilles Romer](https://elearning.iws.th-koeln.de/moodle/user/profile.php?id=2681)
    -   Matriculation Number: 11139919

---

## Project Setup

*This Setup was made for Windows users and will differ if you are using a different OS.    
The recommended Python version used to run this notebook is 3.9.13*

1. Virtual environment

    *This step is required so you can install the required packages without affecting or filling up your global Python installation.*

    ```bash
    python3 -m venv .venv
    source .venv/Scripts/activate
    pip install -r requirements.txt
    ```

2. Git

    *This step is required so unnecessary data from the Notebook is not pushed to the repository. This will automatically remove all metadata and execution counts from the Notebook as soon as you stage your file.*

    1. Download JQ (flexible command-line JSON processor) from [here](https://github.com/stedolan/jq/releases/download/jq-1.6/jq-win64.exe)
    2. Create the following Folder: `C:\Program Files\jq`
    3. Add the following line to your PATH environment variable: `C:\Program Files\jq`
    4. Rename the downloaded JQ file to `jq.exe` and move it to the previously created folder
    5. Add the following lines to your .gitconfig file (usually found in C:\Users\YOUR_USERNAME\.gitconfig)

        ```bash
        [core]
            attributesfile = ~/.gitattributes_global
        [filter "nbstrip_meta"]
            clean = "jq --indent 4 \
                    '(.cells[] | select(has(\"execution_count\")) | .execution_count) = null  \
                    | .metadata = {\"language_info\": {\"name\": \"python\", \"pygments_lexer\": \"ipython3\"}} \
                    | .cells[].metadata = {} \
                    '"
            smudge = cat
            required = true
        [filter "nbstrip_full"]
            clean = "jq --indent 4 \
                    '(.cells[] | select(has(\"outputs\")) | .outputs) = []  \
                    | (.cells[] | select(has(\"execution_count\")) | .execution_count) = null  \
                    | .metadata = {\"language_info\": {\"name\": \"python\", \"pygments_lexer\": \"ipython3\"}} \
                    | .cells[].metadata = {} \
                    '"
            smudge = cat
            required = true
        ```

    6. Create a `.gitattributes_global` file at the same location as your `.gitconfig` file and add the following lines:

        ```bash
        *.ipynb filter=nbstrip_meta
        ```

3. Weights and Biases

    *Weights and Biases is a tool that helps you track your experiments and visualize your results. It is also used to run hyperparameter sweeps.*  
    
    Make sure to create an account on [Weights and Biases](https://wandb.ai/site) and accept the invitation to the Team.  
    Furthermore you will have to add your [API key](https://wandb.ai/settings#dangerzone) to the [scrts.py](./scrts.py) file under `wandb_api_key`.

## Optional Setup

1. Tensorflow GPU support  

    *<span style="color:orange">This step is optional and only required if you want to use a GPU for training.</span>*  
   Follow this setup to activate tensorflow GPU support (Make sure to install the mentioned versions rather than the latest one!): [https://www.tensorflow.org/install/gpu](https://www.tensorflow.org/install/gpu)  
   This tutorial will additionally help you to install CUDA and cuDNN: [https://lifewithdata.com/2022/01/16/how-to-install-tensorflow-and-keras-with-gpu-support-on-windows/](https://lifewithdata.com/2022/01/16/how-to-install-tensorflow-and-keras-with-gpu-support-on-windows/) 

---


## Environment Setup

### Path's

In [ ]:
# Importing the necessary libraries
import os
import sys

# Setting up the path to the root of the repository
path = os.getcwd()
notebookpath = os.path.join(path, 'Exam_WS2223.ipynb')

datapath = os.path.join(path, 'data')
data_testpath = os.path.join(datapath, 'seg_test')
data_trainpath = os.path.join(datapath, 'seg_train')
data_valpath = os.path.join(datapath, 'seg_pred')

### Tensorflow version and GPU availability

In [ ]:
# Importing the libraries
import tensorflow as tf
from tensorflow import keras

# Checking versions and gpu availability
print(f"Tensorflow version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Setting up the GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


### Wandb

In [ ]:
# Importing the libraries
import wandb
import scrts
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
import logging
import absl.logging
import os

# Removing all unnecessary logs
# absl.logging.set_verbosity(absl.logging.ERROR)
logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)
os.environ['WANDB_SILENT'] = 'true'
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['WANDB_NOTEBOOK_NAME'] = notebookpath
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if scrts.wandb_api_key is None:
    print("Please enter your wandb API key in scrts.py")
else:
    wandb.login(key=scrts.wandb_api_key)
# wandb.init(project="Exam", entity="dis21a1_ws22-21_gruppe10")

---
## What are we working with?

### What does the data look like?

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS

# Show metadata of images and folders

# Training data
data_train_length = sum([len(files) for r, d, files in os.walk(data_trainpath)])
# Check if all training images have the same dimensions
for folder in os.listdir(data_trainpath):
    for i, image in enumerate(os.listdir(os.path.join(data_trainpath, folder))):
        if i == 0:
            data_train_imgsize_heigt = Image.open(os.path.join(data_trainpath, folder, image)).height
            data_train_imgsize_width = Image.open(os.path.join(data_trainpath, folder, image)).width
        else:
            if data_train_imgsize_heigt != Image.open(os.path.join(data_trainpath, folder, image)).height or data_train_imgsize_width != Image.open(os.path.join(data_trainpath, folder, image)).width:
                print(f"The train image {folder}/{image} has different dimensions than the others")
                break
print(f"The training data contains {sum([len(files) for r, d, files in os.walk(data_trainpath)])} images with the dimensions {data_train_imgsize_heigt}x{data_train_imgsize_width} pixels")

# Test data
data_val_length = sum([len(files) for r, d, files in os.walk(data_valpath)])
# Check if all test images have the same dimensions
for folder in os.listdir(data_testpath):
    for i, image in enumerate(os.listdir(os.path.join(data_testpath, folder))):
        if i == 0:
            data_test_imgsize_heigt = Image.open(os.path.join(data_testpath, folder, image)).height
            data_test_imgsize_width = Image.open(os.path.join(data_testpath, folder, image)).width
        else:
            if data_test_imgsize_heigt != Image.open(os.path.join(data_testpath, folder, image)).height or data_test_imgsize_width != Image.open(os.path.join(data_testpath, folder, image)).width:
                print(f"The test image {folder}/{image} has different dimensions than the others")
                break
print(f"The test data contains {sum([len(files) for r, d, files in os.walk(data_testpath)])} images with the dimensions {data_test_imgsize_heigt}x{data_test_imgsize_width} pixels")

# Validation data
data_val_length = sum([len(files) for r, d, files in os.walk(data_valpath)])
# Check if all validation images have the same dimensions
for i, image in enumerate(os.listdir(data_valpath)):
    if i == 0:
        data_val_imgsize_heigt = Image.open(os.path.join(data_valpath, image)).height
        data_val_imgsize_width = Image.open(os.path.join(data_valpath, image)).width
    else:
        if data_val_imgsize_heigt != Image.open(os.path.join(data_valpath, image)) or data_val_imgsize_width != Image.open(os.path.join(data_valpath, image)):
            print(f"The validation image {image} has different dimensions than the others")
            break
print(f"The validation data contains {sum([len(files) for r, d, files in os.walk(data_valpath)])} images with the dimensions {data_val_imgsize_heigt}x{data_val_imgsize_width} pixels")

General image size = 150x150

### Loading and normalizing the data

In [ ]:
import random
import numpy as np
classes = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

def load_data(datapath, classes):
    # Only load training and test data
    for dataset in os.listdir(datapath):
        if dataset == "seg_train" or dataset == "seg_test":
            # Load the images
            images = []
            for folder in os.listdir(os.path.join(datapath, dataset)):
                for image in os.listdir(os.path.join(datapath, dataset, folder)):
                    # resize images to 150x150
                    img = Image.open(os.path.join(datapath, dataset, folder, image))
                    img = img.resize((150, 150))
                    images.append([img, folder])
            # Shuffle the images
            random.shuffle(images)
            # Split the images into data(the actual image) and labels
            data = []
            labels = []
            for image in images:
                data.append(image[0])
                labels.append(image[1])
            if dataset == "seg_train":
                train_data = data
                train_labels = labels
            else:
                test_data = data
                test_labels = labels
    return train_data, train_labels, test_data, test_labels

train_data, train_labels, test_data, test_labels = load_data(datapath, classes)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def augment_data(datapath, classes, train_dir=data_trainpath, test_dir=data_testpath):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')

    return train_generator, test_generator

train_generator, test_generator = augment_data(datapath, classes)

### Normalizing the data

In [ ]:
# Vectorizing the labels
def label_vectorization(labels):
    # Convert the labels to numbers
    for i, label in enumerate(labels):
        labels[i] = classes.index(label)
    # One-hot encoding the labels
    from tensorflow.keras.utils import to_categorical
    labels = to_categorical(labels, num_classes=len(classes))
    return labels

train_labels = label_vectorization(train_labels)
test_labels = label_vectorization(test_labels)

# Normalizing the images
def image_normalization(data):
    # Convert the images to arrays
    for i, image in enumerate(data):
        data[i] = np.array(image, dtype="float32")
    # Normalize the images
    data = np.array(data) / 255
    return data

train_data = image_normalization(train_data)
test_data = image_normalization(test_data)

### Checking dataset structure again

In [ ]:
# Checking the dimensions of the data
print(f"The training data contains {len(train_data)} images with the dimensions {train_data[0].shape} pixels")
print(f"The test data contains {len(test_data)} images with the dimensions {test_data[0].shape} pixels")

# Showing training and test data count for each class
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
plt.figure(figsize=(10, 5))
plt.title("Training and test data count for each class")
plt.xlabel("Class")
plt.ylabel("Count")
plt.bar(classes, [train_labels[:, i].sum() for i in range(len(classes))], label="Training data")
plt.bar(classes, [test_labels[:, i].sum() for i in range(len(classes))], label="Test data")
for i in range(len(classes)):
    plt.text(x=classes[i], y=train_labels[:, i].sum(), s=train_labels[:, i].sum(), ha="center")
    plt.text(x=classes[i], y=test_labels[:, i].sum(), s=test_labels[:, i].sum(), ha="center")
plt.legend()
plt.show()

# Showing the proportion each class has in the training and test data in a pie chart
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Training data proportions")
plt.pie([train_labels[:, i].sum() for i in range(len(classes))], labels=classes, autopct="%1.1f%%")
plt.subplot(1, 2, 2)
plt.title("Test data proportions")
plt.pie([test_labels[:, i].sum() for i in range(len(classes))], labels=classes, autopct="%1.1f%%")
plt.show()

# Showing 5 random images from each class
plt.figure(figsize=(15, 10))
plt.suptitle("Image examples from each class")
for i in range(len(classes)):
    plt.subplot(2, 3, i + 1)
    plt.title(classes[i])
    plt.axis("off")
    plt.imshow(train_data[np.random.choice(np.where(train_labels[:, i] == 1)[0])])
plt.show()

---
## Building the model

### Plain classification model

In [ ]:
# Importing the necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model builder
def build_model(layers=[32, 64, 128, 128], dropout=0.5, input_shape=(150, 150, 3)):
    model = Sequential()

    for i, layer in enumerate(layers):
        if i == 0:
            model.add(Conv2D(layer, (3, 3), activation="relu", input_shape=input_shape))
            model.add(MaxPooling2D((2, 2)))
        else:
            model.add(Conv2D(layer, (3, 3), activation="relu"))
            model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(len(classes), activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

# model training
history = build_model().fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

---
### Resnet18 Model

In [ ]:
from classification_models.keras import Classifiers
ResNet18, preprocess_input = Classifiers.get('resnet18')
def build_Resnet_model():
    model = Sequential()

    base = ResNet18(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    model.add(base)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(len(classes), activation='sigmoid'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer = "rmsprop",
        metrics=['acc']
        )

    return model
history = build_Resnet_model().fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

---
### Classification model with data augmentation

In [ ]:
# Importing the necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# model builder
def build_model(layers=[32, 64, 128, 128], dropout=0.5, input_shape=(150, 150, 3)):
    model = Sequential()

    for i, layer in enumerate(layers):
        if i == 0:
            model.add(Conv2D(layer, (3, 3), activation="relu", input_shape=input_shape))
            model.add(MaxPooling2D((2, 2)))
        else:
            model.add(Conv2D(layer, (3, 3), activation="relu"))
            model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(512, activation="relu"))
    model.add(Dense(len(classes), activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

# model training
history = build_model().fit(train_generator, epochs=10, batch_size=32, validation_data=test_generator)